In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


!pip install transformers
from transformers import pipeline

pipe = pipeline("text-classification", model="qanastek/XLMRoberta-Alexa-Intents-Classification")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip -q install datasets
from datasets import load_dataset
from pprint import pprint
model_id = "qanastek/XLMRoberta-Alexa-Intents-Classification"
# make sure to put your own model here
# Before you start make sure to have created an empty repository model in hugging face 🤗 using https://huggingface.co/new
# /
repository_id = "keshavsudo007/abc"

In [3]:
!pip install transformers datasets huggingface_hub tensorboard==2.11
!sudo apt-get install git-lfs --yes
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
from huggingface_hub import HfFolder, notebook_login

import json

# Open the file in read mode
with open("/kaggle/input/surprise-dataset/surprise.data", "r") as file:
    # Initialize an empty list to store selected data
    selected_data = []

    # Read the file line by line
    for line in file:
        # Parse each line as a dictionary
        data_item = json.loads(line)
        
        # Extract only the "indoml_id" and "utt" fields
        selected_item = {
            "indoml_id": data_item["indoml_id"],
            "utt": data_item["utt"]
        }

        # Append the selected data to the list
        selected_data.append(selected_item)

# 'selected_data' now contains only the "indoml_id" and "utt" fields from the file
dataset1=pd.DataFrame(selected_data)

with open("/kaggle/input/surprise-dataset/surprise.solution", "r") as file2:
    # Initialize an empty list to store selected data
    selected_data2 = []

    # Read the file line by line
    for line in file2:
        # Parse each line as a dictionary
        data_item = json.loads(line)
        
        # Extract only the "indoml_id" and "utt" fields
        selected_item = {
            "indoml_id": data_item["indoml_id"],
            "intent": data_item["intent"]
        }

        # Append the selected data to the list
        selected_data2.append(selected_item)

# 'selected_data' now contains only the "indoml_id" and "utt" fields from the file
dataset2=pd.DataFrame(selected_data2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 51.0 MB/s eta 0:00:0000:01:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 41.3 MB/s eta 0:00:00
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.1
    Uninstalling tensorboard-data-server-0.7.1:
      Successfully uninstalled tensorboard-data-server-0.7.1
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.0.0
    Uninstalling google-auth-oauthlib-1.0.0:
      Successfully uninstalled google-auth-oauthlib-1.0.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [4]:
dataset1.shape

(2248, 2)

In [5]:
dataset2.shape

(2248, 2)

In [6]:

# Merge the two DataFrames based on the "indoml_id" column
merged_df = dataset1.merge(dataset2, on="indoml_id", how="inner")

# Create a new DataFrame with "indoml_id," "utt," and "intent" columns
new_dataset = merged_df[["indoml_id", "utt", "intent"]]

# You can save the new dataset to a CSV file if needed
new_dataset.to_csv("combined_dataset.csv", index=False)


In [7]:
new_dataset.shape

(2248, 3)

In [8]:
new_dataset.head(20)

,indoml_id,utt,intent
0,surprise|11109,Can I make a reservation at Buffalo Wild Wings?,accept reservations
1,surprise|11051,Can I book a table for tonight at Bella Vita?,accept reservations
2,surprise|11021,Does the Cheesecake Factory take reservations?,accept reservations
3,surprise|11047,Do they take reservations at Outback Steakhous...,accept reservations
4,surprise|11004,does chili's take reservations?,accept reservations
5,surprise|11097,Can I make a reservation at The French Laundry...,accept reservations
6,surprise|10998,Can I check availability and make a reservatio...,accept reservations
7,surprise|11050,Can you tell me if Carrabba's in Houston accep...,accept reservations
8,surprise|11035,Do you have any availability for next weekend ...,accept reservations
9,surprise|11030,Does Buffalo Wild Wings take reservations for ...,accept reservations


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Assuming 'new_dataset' contains "indoml_id," "utt," and "intent" columns

# Create empty DataFrames for the training and validation datasets
train_dataset = pd.DataFrame(columns=["indoml_id", "utt", "intent"])
val_dataset = pd.DataFrame(columns=["indoml_id", "utt", "intent"])

# Get unique intent labels from the 'new_dataset'
unique_intents = new_dataset["intent"].unique()

# Split the data for each intent
for intent in unique_intents:
    intent_data = new_dataset[new_dataset["intent"] == intent]
    train_data, val_data = train_test_split(intent_data, test_size=5, random_state=42)  # Adjust the test_size as needed

    # Concatenate the training data to the 'train_dataset'
    train_dataset = pd.concat([train_dataset, train_data], ignore_index=True)

    # Concatenate the validation data to the 'val_dataset'
    val_dataset = pd.concat([val_dataset, val_data], ignore_index=True)

# Print the resulting train and validation datasets
print("Train Dataset:")
print(train_dataset.head())

print("\nValidation Dataset:")
print(val_dataset.head())
train_dataset.to_csv("train_dataset.csv", index=False)
val_dataset.to_csv("val_dataset.csv", index=False)

Train Dataset:
        indoml_id                                                utt  \
0  surprise|11035  Do you have any availability for next weekend ...   
1  surprise|11021     Does the Cheesecake Factory take reservations?   
2  surprise|11051      Can I book a table for tonight at Bella Vita?   
3  surprise|11039  Does the Pizza Parlor accept reservations for ...   
4  surprise|11004                    does chili's take reservations?   

                intent  
0  accept reservations  
1  accept reservations  
2  accept reservations  
3  accept reservations  
4  accept reservations  

Validation Dataset:
        indoml_id                                                utt  \
0  surprise|11030  Does Buffalo Wild Wings take reservations for ...   
1  surprise|11042  Does Olive Garden in San Francisco take reserv...   
2  surprise|11109    Can I make a reservation at Buffalo Wild Wings?   
3  surprise|11139        Can I reserve a table at Alinea in Chicago?   
4  surprise|11097  Ca

In [10]:
from transformers import AutoTokenizer, AutoConfig
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define a function for tokenization and label encoding
def preprocess_dataset(dataset):
    tokenized = tokenizer(
        dataset["utt"].tolist(),
        padding=True,
        truncation=True,
        max_length=256,
        return_tensors="pt",
        return_attention_mask=True,
    )
    labels = dataset["intent"].tolist()
    
    # Change 'intent' column to 'label'
    return {**tokenized, "label": labels}

# Create a copy of the original dataset for reference
original_train_dataset = train_dataset.copy()
original_val_dataset = val_dataset.copy()

# Tokenize and preprocess the datasets
train_dataset = preprocess_dataset(train_dataset)
val_dataset = preprocess_dataset(val_dataset)

# Create a label encoder
label_encoder = LabelEncoder()

# Fit the encoder on the unique intent labels in the training dataset
label_encoder.fit(train_dataset['label'])

# Convert intent labels to integers in all datasets
train_dataset['label'] = label_encoder.transform(train_dataset['label'])
val_dataset['label'] = label_encoder.transform(val_dataset['label'])

# Define the format for each dataset
train_dataset = Dataset.from_dict(train_dataset)
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

val_dataset = Dataset.from_dict(val_dataset)
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Continue with training or evaluation as needed

# Extract the number of classes and their names
num_labels = len(label_encoder.classes_)
class_names = label_encoder.classes_
print(f"Number of labels: {num_labels}")
print(f"Class labels: {class_names}")

# Create an id2label mapping
id2label = {i: label for i, label in enumerate(class_names)}

# Update the model's configuration with the id2label mapping
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})


Number of labels: 150
Class labels: ['accept reservations' 'account blocked' 'alarm' 'application status'
 'apr' 'are you a bot' 'balance' 'bill balance' 'bill due' 'book flight'
 'book hotel' 'calculator' 'calendar' 'calendar update' 'calories'
 'cancel' 'cancel reservation' 'car rental' 'card declined' 'carry on'
 'change accent' 'change ai name' 'change language' 'change speed'
 'change user name' 'change volume' 'confirm reservation' 'cook time'
 'credit limit' 'credit limit change' 'credit score' 'current location'
 'damaged card' 'date' 'definition' 'direct deposit' 'directions'
 'distance' 'do you have pets' 'exchange rate' 'expiration date'
 'find phone' 'flight status' 'flip coin' 'food last' 'freeze account'
 'fun fact' 'gas' 'gas type' 'goodbye' 'greeting' 'how busy'
 'how old are you' 'improve credit score' 'income'
 'ingredient substitution' 'ingredients list' 'insurance'
 'insurance change' 'interest rate' 'international fees'
 'international visa' 'jump start' 'last main

In [11]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install transformers
from transformers import pipeline,Trainer
!pip install accelerate --upgrade
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used

In [12]:
!pip install datasets transformers==4.28.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


In [13]:
!pip install datasets transformers==4.28.0
!pip install evaluate
from transformers import DataCollatorWithPadding

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.0 MB/s eta 0:00:00a 0:00:01


In [14]:
# # Model
# model = RobertaForSequenceClassification.from_pretrained(model_id, config=config,ignore_mismatched_sizes=True)
# from transformers import pipeline,Trainer
# # TrainingArguments
# import evaluate

# metric_acc = evaluate.load("accuracy")
# metric_f1 = evaluate.load("f1")
# eval_steps = 1000
# # We need to define a compute_metric function that is supported by the Trainer output
# # It basically converts the logits to predictions and then calls the metric
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     # print("Predictions: ", predictions)
#     # print("Labels: ", labels)
#     return metric_f1.compute(predictions=predictions, references=labels, average="macro")

# training_args = TrainingArguments(
#     output_dir=repository_id,
#     num_train_epochs=5,
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=32,
#     logging_dir=f"{repository_id}/logs",
#     logging_strategy="steps",
#     logging_steps=10,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     warmup_steps=500,
#     evaluation_strategy="steps",
#     save_strategy="steps",
#     save_steps=eval_steps,
#     eval_steps=eval_steps,
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     greater_is_better=True,
# )

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )
# trainer.train()

In [15]:
# # Model
# model = RobertaForSequenceClassification.from_pretrained(model_id, config=config,ignore_mismatched_sizes=True)
# from transformers import pipeline,Trainer
# # TrainingArguments
# import evaluate

# metric_acc = evaluate.load("accuracy")
# metric_f1 = evaluate.load("f1")
# eval_steps = 1000
# # We need to define a compute_metric function that is supported by the Trainer output
# # It basically converts the logits to predictions and then calls the metric
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     # print("Predictions: ", predictions)
#     # print("Labels: ", labels)
#     return metric_f1.compute(predictions=predictions, references=labels, average="macro")

# training_args = TrainingArguments(
#     output_dir=repository_id,
#     num_train_epochs=5,
#     evaluation_strategy="epoch",
#     logging_dir=f"{repository_id}/logs",
#     logging_strategy="steps",
#     logging_steps=10,
#     learning_rate=5e-5,
#     weight_decay=0.01,
#     warmup_steps=500,
#     save_strategy="epoch",
# )

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
# )
# trainer.train()

In [18]:
# Model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config,ignore_mismatched_sizes=True)
!pip install datasets transformers==4.28.0
from transformers import pipeline,Trainer
# TrainingArguments
eval_steps = 1000

training_args = TrainingArguments(
    output_dir=repository_id,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_steps=eval_steps,
    eval_steps=eval_steps,
    load_best_model_at_end=True,
    greater_is_better=True,
    save_strategy="epoch",
)

# from transformers import DataCollatorWithPadding

# Other imports and code

# Create a DataCollatorWithPadding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=256,
)

# Create the trainer with the data_collator
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
# Start training
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at qanastek/XLMRoberta-Alexa-Intents-Classification and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([60, 768]) in the checkpoint and torch.Size([150, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([60]) in the checkpoint and torch.Size([150]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss
1,4.654400,4.571717
2,3.842600,3.585122
3,2.592600,2.452232
4,1.550200,1.629784
5,1.134300,1.391429


TrainOutput(global_step=940, training_loss=3.025209266581434, metrics={'train_runtime': 13780.0844, 'train_samples_per_second': 0.544, 'train_steps_per_second': 0.068, 'total_flos': 986660268410880.0, 'train_loss': 3.025209266581434, 'epoch': 5.0})

In [19]:
trainer.evaluate()

{'eval_loss': 4.571716785430908,
 'eval_runtime': 310.7238,
 'eval_samples_per_second': 2.414,
 'eval_steps_per_second': 0.303,
 'epoch': 5.0}

In [20]:
import json

# Open the file in read mode
with open("/kaggle/input/final-data/massive_test.data", "r") as file:
    # Initialize an empty list to store selected data
    selected_data = []

    # Read the file line by line
    for line in file:
        # Parse each line as a dictionary
        data_item = json.loads(line)
        
        # Extract only the "indoml_id" and "utt" fields
        selected_item = {
            "indoml_id": data_item["indoml_id"],
            "utt": data_item["utt"]
        }

        # Append the selected data to the list
        selected_data.append(selected_item)

# 'selected_data' now contains only the "indoml_id" and "utt" fields from the file
test_data=pd.DataFrame(selected_data)

# Assuming you have a list of sentences in the "utt" column, you can access them like this:
test_sentences = test_data["utt"].tolist()

# Tokenize the test sentences
tokenized_test_data = tokenizer(test_sentences, padding=True, truncation=True, max_length=256, return_tensors="pt")

# Forward pass through the model to obtain logits
with torch.no_grad():
    model.eval()
    outputs = model(**tokenized_test_data)

# Extract logits from the model's output
logits = outputs.logits

# Apply softmax to obtain class probabilities
probs = torch.softmax(logits, dim=1)

# Get the predicted class (intent) for each sentence
predicted_intents = torch.argmax(probs, dim=1)

# Convert predicted intents to a list
predicted_intents_list = predicted_intents.tolist()

# You can decode intent labels if you have the label-to-class mapping
intent_labels = [label_encoder.classes_[label] for label in predicted_intents_list]

# Now, 'intent_labels' contains the predicted intents for each sentence in the test data
print(intent_labels)

['fun fact', 'report lost card', 'shopping list update', 'fun fact', 'tire pressure', 'directions', 'how busy', 'next holiday', 'change ai name', 'min payment', 'change language', 'change ai name', 'travel alert', 'change ai name', 'update playlist', 'who do you work for', 'roll dice', 'accept reservations', 'payday', 'roll dice', 'accept reservations', 'flip coin', 'where are you from', 'transfer', 'payday', 'redeem rewards', 'car rental', 'cancel reservation', 'repeat', 'jump start', 'timer', 'change ai name', 'report fraud', 'meal suggestion', 'directions', 'who do you work for', 'directions', 'calculator', 'vaccines', 'confirm reservation', 'change ai name', 'rewards balance', 'what are your hobbies', 'timer', 'roll dice', 'text', 'change ai name', 'text', 'thank you', 'ingredients list', 'how busy', 'exchange rate', 'redeem rewards', 'spending history', 'order checks', 'meal suggestion', 'travel alert', 'payday', 'yes', 'who do you work for', 'spending history', 'w2', 'yes', 'shop

In [21]:
test_data["intent"]=intent_labels
cvfile = pd.concat((test_data["indoml_id"], test_data["intent"]), axis=1)
cvfile.to_csv('massive_test.predict', index=False)

In [ ]:
# data_list = []
# import json

# for index, row in test_data.iterrows():
#     # Create a dictionary for each row
#     data_dict = {
#         "indoml_id": row['indoml_id'],
#         "intent": row['intent']
#     }
#     # Append the dictionary to the list
#     data_list.append(data_dict)
# file_name = "massive_test2.predict"

# # Open the file in write mode and write the dictionary as JSON
# with open(file_name, 'w') as json_file:
#     for i in data_list:
#         json.dump(data_list, json_file)
# # Convert the list of dictionaries to a DataFrame
# new_df = pd.DataFrame(data_list)
# # Save the DataFrame to a CSV file
# new_df.to_csv("massive_test3.predict", index=False)